In [38]:
import baostock as bs
import pandas as pd 
import datetime  
import openpyxl

start_dt = '2022-02-09'  
end_dt = datetime.datetime.now().strftime(r"%Y-%m-%d")  

freq = 'd'
code = 'sh.600690'
adjustflag = "3"

#### 登陆系统 ####
def get_data(code, content, start_dt, end_dt, freq, adjf):
    lg = bs.login()
    # 显示登陆返回信息
    print('login respond error_code:'+lg.error_code)
    print('login respond  error_msg:'+lg.error_msg) 

    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
    # 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg 
    rs = bs.query_history_k_data_plus(code,
        content,
        start_date=start_dt,
        end_date=end_dt,
        frequency=freq, adjustflag=adjf) 
    print('query_history_k_data_plus respond error_code:'+rs.error_code)
    print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)

    #### 结果集输出到csv文件 ####   
    # result.to_csv("D:\\history_A_stock_k_data.csv", index=False)
    # print(result)

    #### 登出系统 ####
    bs.logout()  
    return result

In [39]:
start_dt = '2022-02-09'  
end_dt = datetime.datetime.now().strftime(r"%Y-%m-%d")  

freq = 'd'
he_code = 'sh.600690'
jdf_code = 'sz.000725'
adjf = "3" 
content = "date,close"

he = get_data(he_code, content, start_dt, end_dt, freq, adjf) 
jdf = get_data(jdf_code, content, start_dt, end_dt, freq, adjf) 

def select_day(x):
    return x[0:10]  

date_part = pd.DataFrame(he['date'])  #  不含周末 
date_part = date_part.stack().apply(str).apply(select_day).unstack()

date_all = pd.date_range(he['date'].min(), he['date'].max(), freq='D')  # 补全，含周末节假日
date = pd.DataFrame(date_all)   # 转换成df 
date = date.stack().apply(str).apply(select_day).unstack()    # 只保留年月日，不保留分钟
date.columns = ['日期']  

# df_pri = pd.DataFrame(he['close'].values.reshape((-1,1)))   
# df_1 = pd.concat([date_part, df_pri], axis=1) 
  
he.columns = ['日期', '海尔股价']   #  he列更名
jdf.columns = ['日期', '京东方A股价']  #  jdf列更名 

he_all = date.set_index('日期').join(he.set_index('日期'))  
he_all.fillna(method='ffill', inplace=True)    

jdf_all = date.set_index('日期').join(jdf.set_index('日期'))  
jdf_all.fillna(method='ffill', inplace=True)

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!


,京东方A股价
日期,
2022-02-09,5.0100
2022-02-10,5.0000
2022-02-11,5.0000
2022-02-12,5.0000
2022-02-13,5.0000
2022-02-14,4.9300
2022-02-15,4.9200
2022-02-16,4.9300
2022-02-17,4.9100


In [35]:
# print(he.head(20))
# print(jdf.head(20))   

# he.columns = ['日期', '海尔收盘价']  
# he.tail(10)  
he_all.head(20)

,海尔股价
日期,
2022-02-09,27.6800
2022-02-10,26.8100
2022-02-11,26.5500
2022-02-12,26.5500
2022-02-13,26.5500
2022-02-14,25.8900
2022-02-15,26.1300
2022-02-16,26.2000
2022-02-17,26.0100
